In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=1
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSP%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFPSP%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempS = InterfaceData(size=inodes, value=T_bot)
TempF = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hF2S = hF[f2s]
hS = solverS.get_interface_h(tag=interface_tag)
hS2F = hS[s2f]

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

CS1 = hS/(hF2S+hS)
CS2 = hF2S/(hF2S+hS)

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FPSP%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSP%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        TempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve solid ambient temperature
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update solid interface temperature
        TempS.curr[:] = CS1 * abTempS.curr + CS2 * abTempF.curr[f2s]
        # update solid temperature interface
        solverS.update_temperature_interface(
            tag=interface_tag,
            T=TempS.curr
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid ambient temperature, the new one
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update fluid interface temperature
        TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
        # update residual
        TempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(TempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(TempF)
            under_relax.update_solution(TempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=200.


KeyboardInterrupt: 

In [ ]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFPSP%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})